In [59]:
totalValuesIntransctions= []
Transactions = []
c={}
l={}
discarded_transactions={} 
supportCount  = {}


# Reading the txt file

In [60]:
def test():
    resultedFile= []
    totalRows = 0
    
    with open('../data.txt','r') as f: 
        lines = f.readlines()



    for line in lines: 
        if totalRows==0:
            totalRows=line
        else: 
            # updatedFile = line.replace('\t','')
            # updatedFile = updatedFile.replace(' ','')
            # updatedFile = line
            totalValuesIntransctions.append(line[1])
            line = line[4:]
            # print(updatedFile)
            # updatedFile=list(updatedFile.replace('\n',''))

            updatedFile = line.split()
            # updatedFile=list(updatedFile.replace('\n',''))

            resultedFile.append(updatedFile)
        

    return resultedFile, totalRows
    


In [61]:
Transactions,totalRows=test()
print(Transactions)

[['1', '3', '4'], ['2', '3', '5'], ['1', '2', '3', '5'], ['2', '5']]


# Minimum Threshold 

In [62]:
minimum_confidence_percent = 50
minimumSupport = (int(totalRows)*minimum_confidence_percent)/100
print(minimumSupport)

2.0


# Counting the items

In [63]:
from collections import Counter
totalRowsCounter=0
totalItems = Counter()
for line in Transactions:

    if totalRowsCounter==0:
        totalItems=(Counter(line))
    else:
        totalItems=(Counter(line)) + totalItems
        print(totalItems)
    totalRowsCounter = totalRowsCounter+1
    

totalItems=dict(totalItems)
print('C1')
print(totalItems)
print(list(totalItems.values()))


Counter({'3': 2, '2': 1, '5': 1, '1': 1, '4': 1})
Counter({'3': 3, '1': 2, '2': 2, '5': 2, '4': 1})
Counter({'2': 3, '5': 3, '3': 3, '1': 2, '4': 1})
C1
{'2': 3, '5': 3, '1': 2, '3': 3, '4': 1}
[3, 3, 2, 3, 1]


In [64]:

initialSize = 1
totalSize =1
sortedOrder = sorted(totalItems.keys())
c.update({initialSize : [[f] for f in sorted(totalItems.keys())] })

discarded_transactions.update({initialSize:[]})
# print(c)

In [65]:
def count_item(singleItemSet, Transactions):
    counter = 0
    for i in range(len(Transactions)):
        if set(singleItemSet).issubset(set(Transactions[i])):
            counter = counter+1
    return counter

In [66]:
def get_frequent(itemSet, initialTransactions, minimumSupport,discarded_transactions):
    tempL = []
    newDiscardedValue = []
    itemCount = []
    # First need to check if the itemset contains subset of items that was previously discarded
    for i in range(len(itemSet)):
        isDiscarded = False
        # if lenght of discarded items is not 0
        if len(discarded_transactions.keys()) > 0:
            # this loop checks if there is any value in the item that was discarded before 
            for k in discarded_transactions[len(discarded_transactions.keys())]:
                
                if set(k).issubset(set(itemSet[i])):
                    isDiscarded = True
                    break
        if isDiscarded == False:
            itemCounter = count_item(itemSet[i],initialTransactions)
            if itemCounter >= minimumSupport:
                tempL.append(itemSet[i])
                itemCount.append(itemCounter)
            else:
                newDiscardedValue.append(itemSet[i])

    return tempL,itemCount,newDiscardedValue
        


In [67]:
receivedL,receiveditemCount,receivedDiscarededValue  =  get_frequent(c[1],Transactions,minimumSupport,discarded_transactions)
l.update({initialSize:receivedL})
supportCount.update({initialSize:receiveditemCount})
discarded_transactions.update({initialSize:receivedDiscarededValue})

In [68]:
print(receivedL,'candidates above threshold ')
print(receiveditemCount)
print(receivedDiscarededValue)
print(c,'amount of candidates')
print(l)

[['1'], ['2'], ['3'], ['5']] candidates above threshold 
[2, 3, 3, 3]
[['4']]
[['1'], ['2'], ['3'], ['4'], ['5']] amount of candidates
{1: [['1'], ['2'], ['3'], ['5']]}


In [30]:
def join_two_itemsets(firstItem,secondItem,sortedOrder):
    firstItem.sort(key=lambda y: sortedOrder.index(y))
    secondItem.sort(key=lambda j: sortedOrder.index(j))

    for i in range(len(firstItem)):
        if firstItem[i] == secondItem[i]:
            return []
    if sortedOrder.index(firstItem[-1]) < sortedOrder.index(secondItem[-1]):
            return firstItem + [secondItem[-1]]

    return []


In [31]:
def joinItemset(items,initialItem):
    # print(items,initialItem,"i am joinItemset")
    tempC = []
    for i in range(len(items)-1):
        for j in range(i+1,len(items)):
            # print(items[i],items[j],initialItem,"i am joinItemset sending this")

            itOut = join_two_itemsets(items[i],items[j],initialItem)
            if len (itOut)>0:
                tempC.append(itOut)
    return tempC
            

In [116]:
print()

In [32]:
totalSize = initialSize + totalSize
noMoreItems = False
while noMoreItems == False:
    joinedSet = joinItemset(l[totalSize-1],sortedOrder)
    # joinedSet = joinItemset(l[totalSize],sortedOrder)
    # print(joinedSet)
    c.update({totalSize:joinedSet})
    receivedL,receiveditemCount,receivedDiscarededValue = get_frequent(c[totalSize],Transactions,minimumSupport,discarded_transactions)
    discarded_transactions.update({totalSize:receivedDiscarededValue})
    l.update({totalSize:receivedL})
    supportCount.update({totalSize:receiveditemCount})
    totalSize = totalSize+initialSize
    if len(receivedL) <1:
        noMoreItems = True

In [33]:
print(l)


{1: [['1'], ['2'], ['3'], ['5']], 2: [['1', '3'], ['2', '3'], ['2', '5'], ['3', '5']], 3: [['2', '3', '5']], 4: []}
